In [ ]:
"""
This code gets all movable/grabbable objs, their locations at certain times of the day,
and the activities being performed at that time from the HOMER_PLUS dataset.
"""

'\nThis code gets all movable/grabbable objs, their locations at certain times of the day,\nand the activities being performed at that time from the HOMER_PLUS dataset.\n'

In [ ]:
# Download HOMER_PLUS dataset from github
!wget https://github.com/Maithili/HOMER_PLUS/zipball/master.zip


--2024-02-15 16:34:52--  https://github.com/Maithili/HOMER_PLUS/zipball/master.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Maithili/HOMER_PLUS/legacy.zip/main [following]
--2024-02-15 16:34:52--  https://codeload.github.com/Maithili/HOMER_PLUS/legacy.zip/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [            <=>     ] 108.78M  3.89MB/s    in 30s     

2024-02-15 16:35:22 (3.62 MB/s) - ‘master.zip’ saved [114059998]



In [ ]:
# Unzip folder
!unzip /content/master.zip

Archive:  /content/master.zip
006621c263af9a16519461c365b2c15e20e10281
   creating: Maithili-HOMER_PLUS-006621c/
  inflating: Maithili-HOMER_PLUS-006621c/.DS_Store  
   creating: Maithili-HOMER_PLUS-006621c/HouseholdA/
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/.DS_Store  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/breakfast_color.jpg  
   creating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/000.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/001.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/002.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/003.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/004.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/005.json  
  inflating: Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/006.json  
  inflating: Maithili-HOMER_PLUS-0066

In [ ]:
import json
import random

# Opening JSON file
number = '009'
file_name = '/content/Maithili-HOMER_PLUS-006621c/HouseholdA/routines_test/' + number + '.json'
f = open(file_name)

In [ ]:
# Returns JSON object as a dictionary
data = json.load(f)

In [ ]:
'''
Iterating through the json to get the names and ids of all of all placable/grabbable objects.
'''
day_arr = {}
time_obj = {}
time_location = {}
id_obj = {}
# Iterate through all graphs in the JSON
for i in range(len(data['graphs'])):
  nodes = data['graphs'][i]['nodes']
  edges = data['graphs'][i]['edges']

  # Iterate through all objects in each node
  for j in nodes:
    # Add objects and their ids to dictionary
    if j['id'] not in id_obj:
      id_obj[j['id']] = j['class_name']

    if ('Furniture' not in j['category']) and (('MOVABLE' in j['properties']) or ('GRABBABLE' in j['properties'])):
      idx = i
      while idx >= len(data['activities']):
        idx -= 1

      if data['times'][i] in time_obj:
        time_obj[data['times'][i]][j['id']] = [j['class_name'],data['activities'][idx]]#.append((j['id'],j['class_name'],data['activities'][idx]))
      else:
        time_obj[data['times'][i]] = {j['id']:[j['class_name'],data['activities'][idx]]} #[(j['id'],j['class_name'],data['activities'][idx])]

  # Iterate through all edges to get current location of objects
  for k in edges:
    from_ids = list(time_obj[data['times'][i]].keys())
    if (data['times'][i] in time_obj) and (k["from_id"] in from_ids) and (k["relation_type"] != "CLOSE"):
      time_obj[data['times'][i]][k["from_id"]].append(str(k["to_id"]))
      time_obj[data['times'][i]][k["from_id"]].append(id_obj[k["to_id"]])


In [ ]:
arr = []
for time in time_obj:
  for obj_id in time_obj[time]:
    activity = time_obj[time][obj_id][1]
    room_ids = [1,41,162,241]
    if (len(time_obj[time][obj_id]) > 4):
      if time_obj[time][obj_id][2] in room_ids:
        room_id = time_obj[time][obj_id][2]
        room_name = time_obj[time][obj_id][3]
        surface_id = time_obj[time][obj_id][4]
        surface_name = time_obj[time][obj_id][5]
      else:
        surface_id = time_obj[time][obj_id][2]
        surface_name = time_obj[time][obj_id][3]
        room_id = time_obj[time][obj_id][4]
        room_name = time_obj[time][obj_id][5]
    else:
      room_id = time_obj[time][obj_id][2]
      room_name = time_obj[time][obj_id][3]
      surface_id = float('inf')
      surface_name = ''
    arr.append({'Time':time,'Obj_ID':obj_id,'Obj_name':id_obj[obj_id],'Room_ID':room_id,'Room_name':room_name,'Surface_ID':surface_id,'Surface_name':surface_name,'Activity':activity})


In [ ]:
import csv

field_names= ['Time', 'Obj_ID', 'Obj_name', 'Room_ID', 'Room_name', 'Surface_ID', 'Surface_name', 'Activity']
name = 'test' + number + '.csv'
with open(name,'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=field_names)
  writer.writeheader()
  writer.writerows(arr)